<a href="https://colab.research.google.com/github/lchutny/MortgageCalcs/blob/main/Copy_of_MortgageCalc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install numpy_financial

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
from os import listdir, fspath
from os.path import isfile, join
import numpy_financial as npf
from datetime import date
from collections import OrderedDict
from dateutil.relativedelta import *

print (f"Python version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Numpy Financial Version {npf.__version__}")

Python version: 3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
Pandas Version: 1.1.5
Numpy version: 1.19.5
Numpy Financial Version 1.0.0


# Input Data

In [ ]:
def calc_r(postedrate, periodsperyear):
    """Calculates the effective annual rate for a Canadian Posted Rate based on Canada's legal Semi Annual Compounding
    and an input number of payment periods per year
    """
    semiannualrate = postedrate/2
    CanAnnualRate=((1+semiannualrate)**2-1)
    reff_perperiod=(1+CanAnnualRate)**(1/periodsperyear)-1  # 
    r = reff_perperiod*periodsperyear # back calculate an effective yearly rate based on the semi annual compounding
    return r

In [ ]:
P = 100000 # principal
postedrate = 0.06 # annual interest rate - as quoted
n = 12 # number of payments per year
r = calc_r(postedrate,n)
t = 25 # amortization period, years
startdate = (date(2021,1,1))
Addl_Principal = 0 # Additional Principal per period

#### Change Rate for Canada - Semi Annual Compounding
From https://vindeep.com/Corporate/InterestRateConversion.aspx and http://www.yorku.ca/amarshal/mortgage.htm   
(In second reference, note error in 2nd line of page, where it should be $r_M = (1.0609)^{1/12}-1$    

Note dox for numpy financial:  
https://numpy.org/numpy-financial/latest/index.html

# Basic Calculations

In [ ]:
r_n = r/n  # interest rate per period
totpymts = n*t # total number of payments
pymt = npf.pmt(r_n,totpymts,P)  # Calculate Payment per period
per = 1 # Payment period to calculate the interest amount
ipmt = npf.ipmt(r_n,per,totpymts,P)
ppmt = npf.ppmt(r_n,per,totpymts,P)
print(f"Principal of ${P}, {n} payments/year, {postedrate*100}% interest as posted, {t} years amortization, payments are ${-pymt:.2f} per period.")
print(f"APR in Canada: for a posted rate of {postedrate*100}%, APR = {r*100:.4f}% and the EAR = ")
print(f"For period {per}, the interest payment is ${-ipmt:.2f} and the principal payment is ${-ppmt:.2f}, not counting for additional payments per period of ${Addl_Principal:.2f}")

Principal of $100000, 12 payments/year, 6.0% interest as posted, 25 years amortization, payments are $639.81 per period.
APR in Canada: for a posted rate of 6.0%, APR = 5.9263%
For period 1, the interest payment is $493.86 and the principal payment is $145.94, not counting for additional payments per period of $0.00


# Build Amortization Table
From https://pbpython.com/amortization-model-revised.html

### Create generator

In [ ]:
def amortize(principal, interest_rate, years, pmt, addl_principal=0, annual_payments=12, start_date=date.today()):
    """Generator to calculate the Amortization table
    This can be used to generate the dataframe
    
    Inputs are:
    principal = Principal amount
    interest_rate = posted annual interest rate
    years = years of amortization, 
    pmt = payment amount per period
    addl_principal = Additional Principal paid per period
    annual_payments = annual number of regular payments
    start_date = 
    start date
    
    Output is Ordered Dictionary of results
    """
        
    # initialize the variables to keep track of the periods and running balances
    p = 1
    beg_balance = principal
    end_balance = principal

    while end_balance > 0:

        # Recalculate the interest based on the current balance
        interest = round(((interest_rate/annual_payments) * beg_balance), 2)

        # Determine payment based on whether or not this period will pay off the loan
        pmt = min(pmt, beg_balance + interest)
        principal = pmt - interest

        # Ensure additional payment gets adjusted if the loan is being paid off
        addl_principal = min(addl_principal, beg_balance - principal)
        end_balance = beg_balance - (principal + addl_principal)

        yield OrderedDict([('Month',start_date),
                           ('Period', p),
                           ('BeginBal', beg_balance),
                           ('Payment', pmt),
                           ('Principal', principal),
                           ('Interest', interest),
                           ('AddtnlPayment', addl_principal),
                           ('EndBal', end_balance)])

        # Increment the counter, balance and date
        p += 1
        start_date += relativedelta(months=1)
        beg_balance = end_balance

### Create Amortization Table

In [ ]:
def AmortTable(principal, interest_rate, years,
                       addl_principal=0, annual_payments=12, start_date=date.today()):
    """
    Calculate the amortization schedule given the loan details as well as summary stats for the loan

    :param principal: Amount borrowed
    :param interest_rate: The annual interest rate for this loan
    :param years: Number of years for the loan
    
    :param annual_payments (optional): Number of payments in a year. DEfault 12.
    :param addl_principal (optional): Additional payments to be made each period. Default 0.
    :param start_date (optional): Start date. Default first of next month if none provided

    :return: 
        schedule: Amortization schedule as a pandas dataframe
        summary: Pandas dataframe that summarizes the payoff information
    """
    
    # Payment stays constant based on the original terms of the loan
    payment = -round(npf.pmt(interest_rate/annual_payments, years*annual_payments, principal), 2)
    
    # Generate the schedule and order the resulting columns for convenience
    schedule = pd.DataFrame(amortize(principal, interest_rate, years, payment,
                                     addl_principal, annual_payments, start_date))
    schedule = schedule[["Period", "Month", "BeginBal", "Payment", "Principal", 
                          "Interest", "AddtnlPayment", "EndBal"]]
    
    # Convert to a datetime object to make subsequent calcs easier
    schedule["Month"] = pd.to_datetime(schedule["Month"])
    
    #Create a summary statistics table
    payoff_date = schedule["Month"].iloc[-1]
    posted_rt = 2*((1+interest_rate/12)**6-1)
    stats = pd.Series([payoff_date, schedule["Period"].count(), round(interest_rate*100,2),
                       round(posted_rt*100,2),years, principal, payment, addl_principal,
                       schedule["Interest"].sum()],
                       index=["Payoff Date", "Num Payments", "Effective Annual Interest Rate, %", 
                              "Posted Annual Rate, %","Amortization Period, Years", "Principal",
                             "Payment", "Additional Payment", "Total Interest"])
    
    return schedule, stats

### Get Results

In [ ]:
sch,stats = AmortTable(principal=P,interest_rate=r,years=t,addl_principal=Addl_Principal,annual_payments=n,start_date=startdate)

sch.head()
#schedule.set_index('Period',inplace=True)

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
0,1,2021-01-01,100000.00,581.6,169.21,412.39,0,99830.79
1,2,2021-02-01,99830.79,581.6,169.91,411.69,0,99660.88
2,3,2021-03-01,99660.88,581.6,170.61,410.99,0,99490.27
3,4,2021-04-01,99490.27,581.6,171.31,410.29,0,99318.96
4,5,2021-05-01,99318.96,581.6,172.02,409.58,0,99146.94


In [ ]:
sch.tail()

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
296,297,2045-09-01,2305.52,581.60,572.09,9.51,0,1733.43
297,298,2045-10-01,1733.43,581.60,574.45,7.15,0,1158.98
298,299,2045-11-01,1158.98,581.60,576.82,4.78,0,582.16
299,300,2045-12-01,582.16,581.60,579.20,2.40,0,2.96
300,301,2046-01-01,2.96,2.97,2.96,0.01,0,0.00


The above schedule for 100,000 at 5% amortized over 25 years with $581.60 payments monthly matches the online calculator exactly (see the pdf). So the amortization schedule is being done correctly.

In [ ]:
stats

Payoff Date                          2046-01-01 00:00:00
Num Payments                                         301
Effective Annual Interest Rate, %                   4.95
Posted Annual Rate, %                                  5
Amortization Period, Years                            25
Principal                                         100000
Payment                                            581.6
Additional Payment                                     0
Total Interest                                     74483
dtype: object

# Scenarios

### 1. First Year of Repayment
2021 - no extra payments. Determine principal owing at end of Dec 2021

In [ ]:
P = 449878.87 # principal at start of 2021
postedrate = 0.0194362 # annual interest rate - as quoted
r = calc_r(postedrate)
t = 25 # amortization period, years
n = 12 # number of payments per year
startdate = (date(2021,1,1))
Addl_Principal = 0 # Additional Principal per period fpr 2021
r_n = r/n  # monthly interest rate
totpymts = n*t # total number of payments

In [ ]:
pymt = npf.pmt(r_n,totpymts,P)  # Calculate Monthly Payment
per = 1 # Payment period to calculate the interest amount
ipmt = npf.ipmt(r_n,per,totpymts,P)
ppmt = npf.ppmt(r_n,per,totpymts,P)
print(f"Principal of ${P}, {n} payments/year, {postedrate*100:.4f}% interest as posted, {t} years amortization, required payments are ${-pymt:.2f} per period.")
print(f"Effective Interest Rate in Canada: {postedrate*100:.4f}% interest as posted = {r*100:.4f}% effective interest rate")
print(f"For period {per}, the interest payment is ${-ipmt:.2f} and the principal payment is ${-ppmt:.2f}, not counting for additional payments per period of ${Addl_Principal:.2f}")

Principal of $449878.87, 12 payments/year, 1.9436% interest as posted, 25 years amortization, required payments are $1892.80 per period.
Effective Interest Rate in Canada: 1.9436% interest as posted = 1.9358% effective interest rate
For period 1, the interest payment is $725.73 and the principal payment is $1167.07, not counting for additional payments per period of $0.00


### 2. Second Year of Repayment  
Assumes:
\$250,000 extra available for the prepayments from settlement of 2836 address;  
2021 - no extra payments  
2022 - repayment of 15% of original principal in Month 3  
2022 - double up all payments for Months 3-12  

In [ ]:
Addl_Principal2 = 0.15*P2/12 # additional amount per month
print(Addl_Principal2)

5595.8414999999995


In [ ]:
sch2a,stats2a = AmortTable(principal=P2,interest_rate=r2,years=t2,addl_principal=Addl_Principal2,annual_payments=n2,start_date=startdate2)

In [ ]:
sch2a.head()

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
0,1,2021-02-21,447667.3200,1839.96,1193.18,646.78,5595.8415,440878.2985
1,2,2021-03-21,440878.2985,1839.96,1202.99,636.97,5595.8415,434079.4670
2,3,2021-04-21,434079.4670,1839.96,1212.81,627.15,5595.8415,427270.8155
3,4,2021-05-21,427270.8155,1839.96,1222.65,617.31,5595.8415,420452.3240
4,5,2021-06-21,420452.3240,1839.96,1232.50,607.46,5595.8415,413623.9825


In [ ]:
stats2a

Payoff Date                          2026-05-21 00:00:00
Num Payments                                          64
Effective Annual Interest Rate, %                   1.73
Posted Annual Rate, %                               1.74
Amortization Period, Years                            25
Principal                                         447667
Payment                                          1839.96
Additional Payment                               5595.84
Total Interest                                     21016
dtype: object